In [1]:
# code written and maintained by Probhakar, 5th April, 2020

In [12]:
# importing required libraries
import random
import math
from collections import namedtuple
import numpy as np
import folium
from folium.plugins import MarkerCluster
import pandas as pd
from jinja2 import Template
import re

In [57]:
# initializing folium map and marker cluster
m = folium.Map(location=[20.5937, 78.9629], zoom_start = 5)
mc = MarkerCluster()

In [102]:
m

In [3]:
# getting all location data
df1 = pd.read_csv('from_place_lat_lon.csv')
df2 = pd.read_csv('state_lat_lon.csv')
df3 = pd.read_csv('city_lat_lon.csv')

In [70]:
print(df1.info())
print(df2.info())
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
Unnamed: 0    100 non-null int64
shortname     100 non-null object
lat           100 non-null float64
lon           100 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 5 columns):
States                    36 non-null object
Abbreviation              36 non-null object
Alternate Abbreviation    36 non-null object
Latitude                  36 non-null float64
Longitude                 36 non-null float64
dtypes: float64(2), object(3)
memory usage: 1.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 4 columns):
Unnamed: 0    233 non-null int64
city_name     233 non-null object
lat           233 non-null float64
lon           233 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 7.4+ KB
None

In [4]:
location_data_pool = {} # dict to keep all latitude, longitude data
for index, row in df1.iterrows():
    location_data_pool[row['shortname'].upper()] = [row['lat'], row['lon']]
    
for index, row in df2.iterrows():
    location_data_pool[row['States'].upper()] = [row['Latitude'], row['Longitude']]
    
for index, row in df3.iterrows():
    location_data_pool[row['city_name'].upper()] = [row['lat'], row['lon']]

In [5]:
# importing main processed file
df = pd.read_csv('covid_19_04_04_2020_filtered.csv')

In [26]:
df.info() # observer the csv file

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 16 columns):
Unnamed: 0                                   4113 non-null int64
Patient Number                               4113 non-null object
State Patient Number                         817 non-null object
Date Announced                               3490 non-null object
Status Change Date                           3487 non-null object
Age Bracket                                  798 non-null object
Gender                                       1027 non-null object
Current Status                               3490 non-null object
Nationality                                  425 non-null object
Type of transmission                         1824 non-null object
Contracted from which Patient (Suspected)    918 non-null object
Detected State                               3490 non-null object
Detected District                            2543 non-null object
Detected City                           

In [9]:
# template for marker popup
popup = Template("""<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:500px}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-1wig{font-weight:bold;text-align:left;vertical-align:top}
.tg .tg-zm03{color:#00009b;border-color:#333333;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-fymr{font-weight:bold;border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0pky"><span style="font-weight:bold">Patient Number</span></th>
    <th class="tg-zm03">{{data[0]}}</th>
  </tr>
  <tr>
    <td class="tg-fymr">State Patient Number</td>
    <td class="tg-0pky">{{data[1]}}</td>
  </tr>
  <tr>
    <td class="tg-fymr">Date Announced</td>
    <td class="tg-0pky">{{data[2]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Status Change Date</td>
    <td class="tg-0lax">{{data[3]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Age Bracket</td>
    <td class="tg-0lax">{{data[4]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Gender</td>
    <td class="tg-0lax">{{data[5]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Current Status</td>
    <td class="tg-0lax">{{data[6]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Nationality</td>
    <td class="tg-0lax">{{data[7]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Type of transmission</td>
    <td class="tg-0lax">{{data[8]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Contracted from which Patient (Suspected)</td>
    <td class="tg-0lax">{{data[9]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Detected State</td>
    <td class="tg-0lax">{{data[10]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Detected District</td>
    <td class="tg-0lax">{{data[11]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Detected City</td>
    <td class="tg-0lax">{{data[12]}}</td>
  </tr>
  <tr>
    <td class="tg-1wig">Notes</td>
    <td class="tg-0lax">{{data[13]}}</td>
  </tr>
</table>""")

In [6]:
# defining method to get siightly deviated lat lon
def get_new_lat_lon(latitude, longitude):
    import random
    import math
    deviation1 = random.randint(0,200)
    deviation2 = random.randint(0,200)
    #print(deviation)
    earth = 6378.137  #radius of the earth in kilometer
    pi = math.pi
    m = (1 / ((2 * pi / 360) * earth)) / 1000;  #1 meter in degree
    new_latitude = latitude + (deviation1 * m)
    
    cos = math.cos
    m = (1 / ((2 * pi / 360) * earth)) / 1000;  #1 meter in degree
    new_longitude = longitude + (deviation2 * m) / cos(latitude * (pi / 180));
    
    return((new_latitude, new_longitude))

In [7]:
# method to get the location co-ordinate
def get_location(row):
    try:
        if(not pd.isna(row['Detected City'])):
            lat = location_data_pool[row['Detected City'].upper()][0]
            lon = location_data_pool[row['Detected City'].upper()][1]
            return(get_new_lat_lon(lat, lon))
        else:
            lat = location_data_pool[row['Detected State'].upper()][0]
            lon = location_data_pool[row['Detected State'].upper()][1]
            return(get_new_lat_lon(lat, lon))
    except:
        lat = location_data_pool[row['Detected State'].upper()][0]
        lon = location_data_pool[row['Detected State'].upper()][1]
        return(get_new_lat_lon(lat, lon))

In [22]:
def get_bearing(p1, p2):
    
    '''
    Returns compass bearing from p1 to p2
    
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    Return
    compass bearing of type float
    
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    
    long_diff = np.radians(p2.lon - p1.lon)
    
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)
    
    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) 
        - (np.sin(lat1) * np.cos(lat2) 
        * np.cos(long_diff)))

    bearing = np.degrees(np.arctan2(x, y))
    
    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing
def get_rotation(locations):
    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90
    return(rotation)

In [58]:
# Step-1: will add each patiend with required details to marker cluster
# patient database
patient = {} # patient_id : lat,lon

for index, row in df.iterrows():
    if(not pd.isna(row['Detected State'])):
        pn = str(row['Patient Number'].split('.')[0])# Patient Number
        stpn = str('no data' if(pd.isna(row['State Patient Number'])) else row['State Patient Number'])#State Patient Number
        da = str('no data' if(pd.isna(row['Date Announced'])) else row['Date Announced'])# Date Announced
        scd = str('no data' if(pd.isna(row['Status Change Date'])) else row['Status Change Date'])# Status Change Date
        ag = str('no data' if(pd.isna(row['Age Bracket'])) else row['Age Bracket'])# Age Bracket
        gd = str('no data' if(pd.isna(row['Gender'])) else row['Gender'])# Gender
        cs = str('no data' if(pd.isna(row['Current Status'])) else row['Current Status'])# Current Status
        nlt = str('no data' if(pd.isna(row['Nationality'])) else row['Nationality'])# Nationality
        tt = str('no data' if(pd.isna(row['Type of transmission'])) else row['Type of transmission'])# Type of transmission
        cp = str('no data' if(pd.isna(row['Contracted from which Patient (Suspected)'])) else row['Contracted from which Patient (Suspected)'])# Contracted from which Patient (Suspected)
        ds = str('no data' if(pd.isna(row['Detected State'])) else row['Detected State'])# Detected State
        dd = str('no data' if(pd.isna(row['Detected District'])) else row['Detected District'])# Detected District
        dc = str('no data' if(pd.isna(row['Detected City'])) else row['Detected City'])# Detected City
        nt = str('no data' if(pd.isna(row['Notes'])) else row['Notes'])# Notes
        pptext = popup.render(data = [pn,stpn,da,scd,ag,gd,cs,nlt,tt,cp,ds,dd,dc,nt])
        location = get_location(row) # (lat,lon)
        patient[pn] = location
        # check if hospitalized, make the color red
        if(row['Current Status'].upper() == 'HOSPITALIZED'):
            mc.add_child(folium.CircleMarker(
            location=location,
            radius=15,
            tooltip = pn,
            popup=pptext,
            color='#ff3300',
            fill=True,
            fill_color='#ff3300'))
        # check if decesed, make the color gray
        elif(row['Current Status'].upper() == 'DECEASED'):
            mc.add_child(folium.CircleMarker(
            location=location,
            radius=15,
            tooltip = pn,
            popup=pptext,
            color='#999966',
            fill=True,
            fill_color='#999966'))
        # check if recovered, make the color green
        elif(row['Current Status'].upper() == 'RECOVERED'):
            mc.add_child(folium.CircleMarker(
            location=location,
            radius=15,
            tooltip = pn,
            popup=pptext,
            color='#009900',
            fill=True,
            fill_color='#009900'))

#finally adding mc to map
m = m.add_child(mc)

In [59]:
# individual point plotted completed, now we will plot the transmission of disease
pat = r'P[0-9]+'
for index, row in df.iterrows():
    pn = str(row['Patient Number'].split('.')[0])
    if(not pd.isna(row['Contracted from which Patient (Suspected)'])):
        disease_from = re.findall(pat, row['Contracted from which Patient (Suspected)'])
        if(len(disease_from) > 0):
            for each in disease_from:
                folium.PolyLine(locations=[list(patient[pn]), list(patient[each])], color='blue', tooltip = each + ' to ' + pn).add_to(m)
                rotation = get_rotation(locations=[list(patient[each]), list(patient[pn])])
                folium.RegularPolygonMarker(location=patient[pn], color = 'blue', fill_color='blue', number_of_sides=3, radius=10, rotation=rotation).add_to(m)
                
                

In [61]:
# now at last we need to put travel history
for index, row in df.iterrows():
    pn = str(row['Patient Number'].split('.')[0])
    if(not pd.isna(row['Travelled From which location'])):
        try:
            from_cordinate = location_data_pool[row['Travelled From which location']] # [lat,lon]
            folium.PolyLine(locations=[list(patient[pn]), from_cordinate], color='#a232a8', tooltip = 'travel from '+row['Travelled From which location'],opacity=0.5).add_to(m)
            rotation = get_rotation(locations=[from_cordinate, list(patient[pn])])
            folium.RegularPolygonMarker(location=patient[pn], color = '#a232a8', fill_color='#a232a8', number_of_sides=3, radius=10, rotation=rotation).add_to(m)
        except Exception as e:
            #print(e)
            pass
            

In [60]:
m.save('user_transmission_travel_from.html')